In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler



In [103]:

df = pd.read_excel("Production_data.xlsx")




In [104]:

# Dummy variable for solar_sensor_key
df['solar_sensor_key'] = df['solar_sensor_key'].astype('category')
ssk_dummies = pd.get_dummies(df['solar_sensor_key'], prefix='ssk')
df = pd.concat([df, ssk_dummies], axis=1)
df = df.drop(columns=['solar_sensor_key'])

# Drop individual_inverter_key
df = df.drop(columns=['individual_inverter_key'])
# Target and labels
X = df.drop(columns=['DC_POWER'])
y = df['DC_POWER'].astype('float')


df

,HOUR,MINUTES,PLANT_ID,DC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,ssk_HmiyD2TTLFNqkNe,ssk_iq8k7ZNt4Mwm3w0
0,0,0,4135001,0.0,0.0,6259559.0,25.184,22.858,0.0,1,0
1,0,0,4135001,0.0,0.0,6183645.0,25.184,22.858,0.0,1,0
2,0,0,4135001,0.0,0.0,6987759.0,25.184,22.858,0.0,1,0
3,0,0,4135001,0.0,0.0,7602960.0,25.184,22.858,0.0,1,0
4,0,0,4135001,0.0,0.0,7158964.0,25.184,22.858,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
136471,23,45,4136001,0.0,4157.0,520758.0,23.203,22.536,0.0,0,1
136472,23,45,4136001,0.0,3931.0,121131356.0,23.203,22.536,0.0,0,1
136473,23,45,4136001,0.0,4322.0,2427691.0,23.203,22.536,0.0,0,1
136474,23,45,4136001,0.0,4218.0,106896394.0,23.203,22.536,0.0,0,1


In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Ensure that the indices are aligned after removing rows
X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]

X_test = X_test.dropna()
y_test = y_test.loc[X_test.index]

# Remove rows with infinite or NaN values
X_train = X_train.replace([np.inf, -np.inf], np.nan).dropna()
y_train = y_train.loc[X_train.index]

X_test = X_test.replace([np.inf, -np.inf], np.nan).dropna()
y_test = y_test.loc[X_test.index]

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [106]:
# Create the neural network model
model = Sequential(
    [
        Dense(units=1, input_shape=[X_train_scaled.shape[1]]),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='relu')
        
    ], name="pennwick_model"
)

# Compile the model
model.compile( 
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001 ),
    metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

# Train the model
model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)

# Make predictions
y_pred_nn = model.predict(X_test_scaled)


Epoch 1/100
2730/2730 [==============================] - 17s 6ms/step - loss: 1766865.6250 - mean_squared_error: 1766865.6250 - mean_absolute_error: 634.4949 - val_loss: 247241.1562 - val_mean_squared_error: 247241.1562 - val_mean_absolute_error: 263.2294
Epoch 2/100
2730/2730 [==============================] - 16s 6ms/step - loss: 214772.2500 - mean_squared_error: 214772.2500 - mean_absolute_error: 208.1932 - val_loss: 248439.0000 - val_mean_squared_error: 248439.0000 - val_mean_absolute_error: 229.2444
Epoch 3/100
2730/2730 [==============================] - 18s 7ms/step - loss: 196367.1875 - mean_squared_error: 196367.1875 - mean_absolute_error: 182.9303 - val_loss: 191163.4531 - val_mean_squared_error: 191163.4531 - val_mean_absolute_error: 182.7275
Epoch 4/100
2730/2730 [==============================] - 22s 8ms/step - loss: 188833.5156 - mean_squared_error: 188833.5156 - mean_absolute_error: 177.7699 - val_loss: 186327.3125 - val_mean_squared_error: 186327.3125 - val_mean_absolut

In [107]:
# Calculate RMSE
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))
print("RMSE:", rmse_nn)

RMSE: 422.2637916471617


In [70]:
df

,HOUR,MINUTES,PLANT_ID,DC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,ssk_HmiyD2TTLFNqkNe,ssk_iq8k7ZNt4Mwm3w0
510,6,0,4135001,37.14,0.00,6.259559e+06,24.088,22.207,0.01,1,0
511,6,0,4135001,53.50,0.00,6.183645e+06,24.088,22.207,0.01,1,0
512,6,0,4135001,58.00,0.00,6.987759e+06,24.088,22.207,0.01,1,0
513,6,0,4135001,58.43,0.00,7.602960e+06,24.088,22.207,0.01,1,0
514,6,0,4135001,54.38,0.00,7.158964e+06,24.088,22.207,0.01,1,0
...,...,...,...,...,...,...,...,...,...,...,...
136009,18,30,4136001,19.65,4153.53,5.207545e+05,24.582,24.787,0.01,0,1
136010,18,30,4136001,18.14,3927.67,1.211314e+08,24.582,24.787,0.01,0,1
136011,18,30,4136001,20.27,4318.40,2.427687e+06,24.582,24.787,0.01,0,1
136012,18,30,4136001,19.88,4214.60,1.068964e+08,24.582,24.787,0.01,0,1


In [112]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Create a Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Define a parameter distribution for hyperparameter tuning
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None] + list(range(10, 31)),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5),
    'max_features': [1.0, 'sqrt']
}

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=50, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error', random_state=42)

# Fit the model using the training data
random_search.fit(X_train_scaled, y_train)

# Print the best combination of hyperparameters
print("Best hyperparameters:", random_search.best_params_)

# Train the Random Forest model with the best hyperparameters
best_rf_model = random_search.best_estimator_

# Make predictions using the best Random Forest model
y_pred_best_rf = best_rf_model.predict(X_test_scaled)

# Calculate RMSE
rmse_best_rf = np.sqrt(mean_squared_error(y_test, y_pred_best_rf))
print("Best Random Forest RMSE:", rmse_best_rf)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=1, min_samples_split=9, n_estimators=70; total time=   6.8s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=1, min_samples_split=9, n_estimators=70; total time=   6.9s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=1, min_samples_split=9, n_estimators=70; total time=   7.0s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=3, min_samples_split=8, n_estimators=124; total time=  12.6s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=3, min_samples_split=8, n_estimators=124; total time=  12.7s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=3, min_samples_split=8, n_estimators=124; total time=  12.9s
[CV] END max_depth=19, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=153; total time=  17.6s
[CV] END max_depth=19, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=153; 

In [113]:
feature_importances = best_rf_model.feature_importances_
# Create a DataFrame to visualize the feature importances
importances_df = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances})
importances_df = importances_df.sort_values(by='importance', ascending=False)
print(importances_df)


               feature  importance
7          IRRADIATION    0.300972
6   MODULE_TEMPERATURE    0.217007
8  ssk_HmiyD2TTLFNqkNe    0.118538
9  ssk_iq8k7ZNt4Mwm3w0    0.110378
2             PLANT_ID    0.104540
5  AMBIENT_TEMPERATURE    0.052826
0                 HOUR    0.038477
3          DAILY_YIELD    0.031169
4          TOTAL_YIELD    0.025111
1              MINUTES    0.000981


In [114]:
import joblib

# Save the model to a file
joblib.dump(best_rf_model, 'best_rf_model.pkl')

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Load the model from the file (when needed)
# loaded_rf_model = joblib.load('best_rf_model.pkl')
# Load the scaler (when needed)
# loaded_scaler = joblib.load('scaler.pkl')

['scaler.pkl']